![shimoku.png](img/shimoku.png)

The aim of this project is to use the Shimoku interface to create a dashboard that can answer these three questions related to our movies and shows datasets:<br>
1. How have genre trends evolved over the years?<br>
2. How does each platform manage movies?<br>
3. What impact do actors or directors have on the evaluation of movies?<br>

In [4]:
from os import getenv
import shimoku_api_python as Shimoku

import pandas as pd
import os
from collections import Counter

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV

import utils

## Let´s quicly take a look at some info about the datasets

In [7]:
# Set working directory and import datasets
os.chdir(r'your_path/repo/Shimoku/')

df_credits = pd.read_csv('data/all_credits.csv',
                         index_col=0)
df_credits.head()

,person_id,id,name,character,role
0,25020,tm87233,James Stewart,George Bailey,ACTOR
1,26190,tm87233,Donna Reed,Mary Hatch,ACTOR
2,25653,tm87233,Lionel Barrymore,Mr. Potter,ACTOR
3,13301,tm87233,Thomas Mitchell,Uncle Billy,ACTOR
4,34171,tm87233,Henry Travers,Clarence,ACTOR


In [8]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356521 entries, 0 to 35286
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   person_id  356521 non-null  int64 
 1   id         356521 non-null  object
 2   name       356521 non-null  object
 3   character  319081 non-null  object
 4   role       356521 non-null  object
dtypes: int64(1), object(4)
memory usage: 16.3+ MB


In [9]:
df_nulls_percentage = round(df_credits.isnull().sum() / len(df_credits) * 100)
df_nulls_percentage

person_id     0.0
id            0.0
name          0.0
character    11.0
role          0.0
dtype: float64

So, 11% of character names are nulls, let´s not forget that when creating charts

In [10]:
df_titles = pd.read_csv('data/all_titles.csv',
                        index_col=0)
df_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,streaming
0,tm87233,It's a Wonderful Life,MOVIE,A holiday favourite for generations... George...,1946,PG,130,"['drama', 'romance', 'comedy', 'family', 'fant...",['US'],NaN,tt0038650,8.6,470551.0,27.860,8.259,amazon
1,ts20945,The Three Stooges,SHOW,The Three Stooges were an American vaudeville ...,1934,TV-PG,19,"['comedy', 'family']",['US'],26.0,tt0850645,8.5,1172.0,15.424,7.600,amazon
2,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,107,"['comedy', 'drama', 'action', 'war', 'western'...",['US'],NaN,tt0017925,8.1,93558.0,10.345,8.014,amazon
3,tm82253,The Best Years of Our Lives,MOVIE,It's the hope that sustains the spirit of ever...,1946,NaN,172,"['drama', 'romance', 'war']",['US'],NaN,tt0038351,8.1,66794.0,15.819,7.843,amazon
4,tm38206,Whoopee!,MOVIE,Western sheriff Bob Wells is preparing to marr...,1930,NaN,93,"['comedy', 'romance', 'western']",['US'],NaN,tt0021549,6.4,1103.0,1.174,7.000,amazon


In [11]:
df_titles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24736 entries, 0 to 2631
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    24736 non-null  object 
 1   title                 24736 non-null  object 
 2   type                  24736 non-null  object 
 3   description           24544 non-null  object 
 4   release_year          24736 non-null  int64  
 5   age_certification     12423 non-null  object 
 6   runtime               24736 non-null  int64  
 7   genres                24736 non-null  object 
 8   production_countries  24736 non-null  object 
 9   seasons               6584 non-null   float64
 10  imdb_id               22787 non-null  object 
 11  imdb_score            22123 non-null  float64
 12  imdb_votes            22060 non-null  float64
 13  tmdb_popularity       24009 non-null  float64
 14  tmdb_score            21914 non-null  float64
 15  streaming           

In [12]:
titles_nulls_percentage = round(df_titles.isnull().sum() / len(df_titles) * 100)
titles_nulls_percentage

id                       0.0
title                    0.0
type                     0.0
description              1.0
release_year             0.0
age_certification       50.0
runtime                  0.0
genres                   0.0
production_countries     0.0
seasons                 73.0
imdb_id                  8.0
imdb_score              11.0
imdb_votes              11.0
tmdb_popularity          3.0
tmdb_score              11.0
streaming                0.0
dtype: float64

Many missing information on this dataset

In [13]:
df_titles[['genres']].iloc[0][0]

"['drama', 'romance', 'comedy', 'family', 'fantasy']"

In [14]:
# I´ve noticed the genres and production_countries column type it´s a bit weird so let´s fix it first
df_titles[['genres', 'production_countries']] = df_titles[['genres', 'production_countries']].applymap(lambda x: x.replace('[','').replace(']', '').replace("'", '').replace(' ', '').split(','))

## Connect to our Shimoku account

In [10]:
access_token = getenv('SHIMOKU_TOKEN')
universe_id = getenv('UNIVERSE_ID')
workspace_id = getenv('WORKSPACE_ID')

s = Shimoku.Client(
    access_token=access_token,
    universe_id=universe_id,
    verbosity='INFO',
)

s.set_workspace(uuid=workspace_id)

2023-09-06 20:56 | INFO | Starting execution: set_workspace
2023-09-06 20:56 | INFO | Finished execution: set_workspace, elapsed time: 1457.84 ms


## 1. How have genre trends evolved over the years?

Two area charts will be created: one for movies and one for shows, so we´ll be able to see how genres have evolved over the years on each one.<br>
I have noted that, at least on this chart, only 8 different features can be compared, so let´s see first wich are the most important genres and let´s plot them.

In [11]:
# Filter by type
df_movies = df_titles[df_titles['type'] == 'MOVIE'].reset_index()
df_shows= df_titles[df_titles['type'] == 'SHOW'].reset_index()

# Get the 8 most common genres
main_movie_genres, main_show_genres = utils.common_genres(df_movies, df_shows)

print('Main movie genders: \n', main_movie_genres, '\nMain show genders: \n', main_show_genres)

Main movie genders: 
 ['drama', 'comedy', 'thriller', 'action', 'romance', 'crime', 'horror', 'documentation'] 
Main show genders: 
 ['drama', 'comedy', 'animation', 'action', 'scifi', 'documentation', 'family', 'crime']


### Movie & Show Genders data

In [12]:
# Get a list of dictionaries with the genres percentage per year
movie_genres_data = utils.get_movie_genres_data(df_movies, main_movie_genres)
show_genres_data = utils.get_show_genres_data(df_shows, main_show_genres)

print('Movie dictionary example:\n', movie_genres_data[-1])

Movie dictionary example:
 {'drama': 0.22, 'thriller': 0.14, 'documentation': 0.15, 'romance': 0.09, 'comedy': 0.21, 'horror': 0.02, 'crime': 0.07, 'action': 0.1, 'year': '2023'}


### Upload Charts

So the design for this part will consist on a board called Genres, with a menu called 'Over the years'.<br>
Inside it, there will be two tabs: one with a stacked area chart with movie genres evolution and the other for shows

In [13]:
# Set board name and menu path
s.set_board('Genres')
s.set_menu_path('Over the years',)

# Then, organize the charts on two tabs called 'Movies' and 'Shows'
s.plt.set_tabs_index(('Gender Charts', 'Movies'), order=0)

s.plt.stacked_area(
    data=movie_genres_data, x="year", order=0,
    title='Movie genres over the years',
    x_axis_name='Years',
    y_axis_name='Genres %'
)

s.plt.change_current_tab('Shows')
s.plt.stacked_area(
    data=show_genres_data, x="year", order=0,
    title='Show genres over the years',
    x_axis_name='Years',
    y_axis_name='Genres %'
)

s.plt.pop_out_of_tabs_group()

2023-09-03 13:30 | INFO | Starting execution: set_board
2023-09-03 13:30 | INFO | Retrieved board Genres with id db826b42-086a-4478-9960-f756ad7decfb
2023-09-03 13:30 | INFO | Finished execution: set_board, elapsed time: 720.48 ms
2023-09-03 13:30 | INFO | Starting execution: set_menu_path
2023-09-03 13:30 | INFO | Created menu path Over the years with id b4e2b508-7317-493e-b1fa-b3b1c3bd4870
2023-09-03 13:30 | INFO | Menu path Over the years added to board Genres
2023-09-03 13:30 | INFO | Finished execution: set_menu_path, elapsed time: 4034.54 ms
2023-09-03 13:30 | INFO | Starting execution: set_tabs_index
2023-09-03 13:30 | INFO | Created tabs group Gender Charts with id cb61ac87-2335-4e9b-a0e7-04d2eb302b2a
2023-09-03 13:30 | INFO | Finished execution: set_tabs_index, elapsed time: 2112.82 ms
2023-09-03 13:30 | INFO | Updated tab groups and modals
2023-09-03 13:30 | INFO | Starting execution: stacked_area_chart
2023-09-03 13:30 | INFO | Created EChart with id 05bedc2d-1a72-4818-9f51-

And this is the result:

![genres.png](img/movie_genres.png)
![genres.png](img/show_genres.png)

## 2. How does each platform manage movies?

Now, we are going to crate different charts for each platforms. We´ll do that with a for loop iterating over each platform.

In [14]:
s.set_board('Platforms')
s.set_menu_path('Comparison')

2023-09-03 13:31 | INFO | Starting execution: set_board
2023-09-03 13:31 | INFO | Retrieved board Platforms with id 82e2a00d-91cb-40f4-bdab-fa48ca5f1b88
2023-09-03 13:31 | INFO | Finished execution: set_board, elapsed time: 118.58 ms
2023-09-03 13:31 | INFO | Starting execution: set_menu_path
2023-09-03 13:31 | INFO | Created menu path Comparison with id c59da776-6451-4189-a149-e7e4d8053334
2023-09-03 13:31 | INFO | Menu path Comparison added to board Platforms
2023-09-03 13:31 | INFO | Finished execution: set_menu_path, elapsed time: 1636.99 ms


In [18]:
platforms = sorted(df_titles.streaming.unique())
order = [0 if i == 0 else 6*i for i in range(len(platforms))]     #This is for organising the charts later on
colors_dict = {'amazon':'#00A8E1', 'disney':'#0063E1', 'hbo':'#941de8', 'hulu':'#1CE783', 'netflix':'#E50914'}     #HEX color of each platform

for i, platform in zip(order, platforms):
    # Get number of movies on the platform
    df_platform_movies = df_titles[(df_titles['streaming'] == platform) & (df_titles['type'] == 'MOVIE')]
    num_platform_movies = df_platform_movies.shape[0]
    
    # Mean movie scores
    mean_score_imdb = round(df_platform_movies['imdb_score'].mean(), 1)
    mean_score_tmdb = round(df_platform_movies['tmdb_score'].mean(), 1)
    
    # Get the genres distribution
    platform_genres_data = utils.get_platform_genres_data(df_platform_movies)
    
    # Get the countries distribution
    platform_countries_data = utils.get_platform_countries_data(df_platform_movies)
    
    # Upload the different charts
    # Platform title
    html = utils.html_style(platform)
    s.plt.html(html, order=i)
    
    # Number of movies indicator
    indicator = {
        'title':'Number of movies',
        'value':num_platform_movies,
        "align" : 'left',
        'variant':'topColor'
    }
    s.plt.indicator(
        order=i+1,
        data=indicator,
        rows_size=2, cols_size=3,
        vertical=True
    )

    # Movies scores
    s.plt.speed_gauge(
        title='IMDB Movies Mean Score', name='',
        value=mean_score_imdb, order=i+2,
        min_value=0, max_value=10,
        rows_size=2, cols_size=3,
        option_modifications={'color':colors_dict[platform]}
    )

    s.plt.speed_gauge(
        title='TMDB Movies Mean Score', name='',
        value=mean_score_tmdb, order=i+3,
        min_value=0, max_value=10,
        rows_size=2, cols_size=3,
        option_modifications={'color':colors_dict[platform]}
    )
    
    # Genres pie
    s.plt.pie(
        data=platform_genres_data, names='genre', values='value',
        order=i+4, rows_size=2, cols_size=6,
        title='Movies per genre'
    )

    #Countries pie
    s.plt.pie(
        data=platform_countries_data, names='country', values='value',
        order=i+5, rows_size=2, cols_size=6,
        title='Movies per country'
    )

2023-09-03 13:37 | INFO | Starting execution: html
2023-09-03 13:37 | INFO | No changes needed for HTML at 0
2023-09-03 13:37 | INFO | Finished execution: html, elapsed time: 2.53 ms
2023-09-03 13:37 | INFO | Starting execution: indicator
2023-09-03 13:37 | INFO | Starting execution: create indicator
2023-09-03 13:37 | INFO | No changes needed for Indicator at 1
2023-09-03 13:37 | INFO | Finished execution: create indicator, elapsed time: 10.21 ms
2023-09-03 13:37 | INFO | Finished execution: indicator, elapsed time: 123.21 ms
2023-09-03 13:37 | INFO | Starting execution: speed_gauge_chart
2023-09-03 13:37 | INFO | Deleted data set with name 48d1b998-cf8c-4c07-a512-f2376e0eb485
2023-09-03 13:37 | INFO | Created data set with id 4a3e5e2f-1bda-411c-83c5-21f5b677a08f and name 48d1b998-cf8c-4c07-a512-f2376e0eb485
2023-09-03 13:37 | INFO | Deleted 0 component data set links from component at 2
2023-09-03 13:37 | INFO | Created 1 component data set links for component 2
2023-09-03 13:37 | IN

Result:

![platforms_1](img/platforms_1.png)
![platforms_2](img/platforms_2.png)

## 3. What impact do actors or directors have on the evaluation of movies?

To find out what impact each variable has on the evaluation of a movie, we are going to develop a ML model that will be able to tell us the weight of each variable

In [28]:
# First merge both dataframes so we have all the information for each name
df_merged = pd.merge(left=df_titles, right=df_credits, on='id', how='inner').reset_index(drop=True)
df_merged_mov = df_merged[df_merged['type'] == 'MOVIE'].reset_index(drop=True)

# Drop those columns that will not actually provide valuable information for the target variable
df_model = df_merged_mov.drop(['id','description','age_certification','seasons', 'imdb_id','imdb_votes','tmdb_popularity','tmdb_score','person_id','title','type','character'],
                              axis=1)

### Feature engineering

#### Step 1: create runtime groups of 30 mins

In [29]:
print('Max duration: ',df_merged_mov.runtime.max())
print('Min duration: ',df_merged_mov.runtime.min())

Max duration:  328
Min duration:  0


In [30]:
runtime_groups = utils.create_runtime_groups(df_model)
df_model['runtime_groups'] = runtime_groups

Counter(runtime_groups)

Counter({'120_150': 68027,
         '90_120': 162531,
         '60_90': 79054,
         '30_60': 11108,
         '>=180': 3391,
         '0_30': 3581})

#### Step 2: create dummies columns for main countries

In [31]:
top10_countries = utils.top10_countries(df_model)
top10_countries

['US', 'GB', 'IN', 'CA', 'FR', 'DE', 'JP', 'ES', 'IT', 'AU']

In [32]:
# Create a dictionary to save each country appearance
dummies_dict_c = utils.countries_dummy_dict(df_model, top10_countries)

# Add the new dummies columns to the dataframe
for new_col_c in dummies_dict_c.keys():
    df_model[new_col_c] = dummies_dict_c[new_col_c]

#### Step 3: create dummies columns for main genres

In [33]:
top10_genres = utils.top10_genres(df_model)
top10_genres

['drama',
 'comedy',
 'thriller',
 'action',
 'romance',
 'crime',
 'family',
 'fantasy',
 'european',
 'scifi']

In [34]:
# Create a dictionary to save each genre appearance
dummies_dict_g = utils.genres_dummy_dict(df_model, top10_genres)

# Add the new dummies columns to the dataframe
for new_col_g in dummies_dict_g.keys():
    df_model[new_col_g] = dummies_dict_g[new_col_g]

#### Step 3: go for the model

In [71]:
df_model_clean = df_model.copy()
# Remove more columns and rows with NA on IMDB column
df_model_clean.drop(['runtime', 'genres', 'production_countries', 'streaming','role'],
                    inplace=True,
                    axis=1)

df_model_clean.dropna(subset=['imdb_score'],
                      inplace=True)

In [72]:
# Label encoder
encoder = LabelEncoder()

columns_encoder = ['runtime_groups', 'name']
df_model_clean[columns_encoder] = df_model_clean[columns_encoder].apply(encoder.fit_transform)

In [73]:
# Set the target and the explanatory variables
X = df_model_clean.drop('imdb_score', axis = 1).values
target = df_model_clean['imdb_score'].values

In [74]:
# Split data in train and test

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state=42)

In [76]:
# The Random Forest Regressor is the chosen algorithm  
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print('r2', round(r2,2), '\nmse', round(mse, 2), sep=':')

r2:0.77:
mse:0.31


In [77]:
# This algorithm has a feature that can tell us the importance (%) of each variable
importances = model.feature_importances_

feature_names = df_model_clean.drop('imdb_score', axis = 1).columns
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

print(feature_importances)

                  Feature  Importance
0            release_year    0.262787
1                    name    0.160481
2          runtime_groups    0.126625
14            dummy_drama    0.037102
16         dummy_thriller    0.035506
15           dummy_comedy    0.034541
24     dummy_OTHER_genres    0.030982
17           dummy_action    0.030287
23            dummy_scifi    0.027013
3                dummy_US    0.026665
18          dummy_romance    0.026431
20           dummy_family    0.025319
19            dummy_crime    0.025154
21          dummy_fantasy    0.024335
5                dummy_IN    0.022143
4                dummy_GB    0.019652
13  dummy_OTHER_countries    0.019351
6                dummy_CA    0.013771
22         dummy_european    0.013656
8                dummy_DE    0.008965
7                dummy_FR    0.007323
9                dummy_JP    0.006674
11               dummy_IT    0.006408
12               dummy_AU    0.004795
10               dummy_ES    0.004033


In [106]:
# Save trained model to a pickle
import pickle

with open('models/trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# To load it
# with open('models/trained_model', 'rb') as f:
#     model = pickle.load(f)

We could keep trying to improve the model: maybe removing variables, creating new ones, doing a gridsearch... The R2 and mse are not bad so let´s move on to the next part

### Create and upload the charts

In [78]:
s.set_board('IMDB Score')
s.set_menu_path('Importances')

2023-09-06 23:17 | INFO | Starting execution: set_board
2023-09-06 23:17 | INFO | Created board IMDB Score with id 65983d28-c177-41d2-a497-47ceec3ab99e
2023-09-06 23:17 | INFO | Finished execution: set_board, elapsed time: 3673.45 ms
2023-09-06 23:17 | INFO | Starting execution: set_menu_path
2023-09-06 23:17 | INFO | Created menu path Importances with id af9f65fa-099e-4265-93d4-785a7a9afb10
2023-09-06 23:18 | INFO | Menu path Importances added to board IMDB Score
2023-09-06 23:18 | INFO | Finished execution: set_menu_path, elapsed time: 5891.97 ms


In [102]:
# Indicators with the top 3 variables for imdb score prediction
html_1 = '''<style>
        .title-text {
        font-family: Arial;
        font-size: 50px;
        color: #000000;
        }
        </style>
        <p class="title-text">TOP 3 Important Variables to Determine IMDB Movie Score</p>
        '''
s.plt.html(html_1, order=0)

html_2 = '''<style>
        .subtitle-text {
        font-family: Arial;
        font-size: 30px;
        color: #000000;
        }
        </style>
        <p class="subtitle-text">1. Release year</p>
        <br>
        <p class="subtitle-text">2. Cast & Director</p>
        <br>
        <p class="subtitle-text">3. Run time</p>
        '''
s.plt.html(html_2, order=1)

# Bar plot with importances
data = [
    {'var': 'Release Year', 'variable weight': round(feature_importances.Importance[0] * 100)},
    {'var': 'Cast & Director', 'variable weight': round(feature_importances.Importance[1] * 100)},
    {'var': 'Run time', 'variable weight': round(feature_importances.Importance[2] * 100)},
    {'var': 'Genre: Drama', 'variable weight': round(feature_importances.Importance[3] * 100)},
    {'var': 'Genre: Thriller', 'variable weight': round(feature_importances.Importance[4] * 100)},
    {'var': 'Genre: Comedy', 'variable weight': round(feature_importances.Importance[5] * 100)},
    {'var': 'Genre: Other', 'variable weight': round(feature_importances.Importance[6] * 100)},
    {'var': 'Genre: Action', 'variable weight': round(feature_importances.Importance[7] * 100)},
    {'var': 'Genre: Scifi', 'variable weight': round(feature_importances.Importance[8] * 100)},
    {'var': 'Country: US', 'variable weight': round(feature_importances.Importance[9] * 100)},
    {'var': 'Other variables', 'variable weight': round(feature_importances.Importance[10:].sum() * 100)}
]

s.plt.bar(
    data=data, order=2,
    x='var', rows_size=2, cols_size=12,
    y_axis_name='Var importance (%)'
)

2023-09-06 23:58 | INFO | Starting execution: html
2023-09-06 23:58 | INFO | No changes needed for HTML at 0
2023-09-06 23:58 | INFO | Finished execution: html, elapsed time: 1.84 ms
2023-09-06 23:58 | INFO | Starting execution: html
2023-09-06 23:58 | INFO | No changes needed for HTML at 1
2023-09-06 23:58 | INFO | Finished execution: html, elapsed time: 1.74 ms
2023-09-06 23:58 | INFO | Starting execution: bar_chart
2023-09-06 23:58 | INFO | Deleted data set with name 17e1fecf-064e-4810-ade0-8a153d8d891f
2023-09-06 23:58 | INFO | Created data set with id 2b3a9268-3daf-46c3-9211-de046d405fcc and name 17e1fecf-064e-4810-ade0-8a153d8d891f
2023-09-06 23:58 | INFO | Deleted 0 component data set links from component at 2
2023-09-06 23:58 | INFO | Created 2 component data set links for component 2
2023-09-06 23:58 | INFO | Updated EChart at 2
2023-09-06 23:58 | INFO | Finished execution: bar_chart, elapsed time: 5715.88 ms


Result:

![IMDB](img/imdb_score.png)

In my personal opinion, Shimoku offers a wide variety of possibilities to show data, with different charts, indicators, layouts... With the possibility of automating all these processes through python. I found it quite interesting and I guess that if you spend more time to discover all its features you could end up with an even cooler dashboard.